In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install --upgrade transformers
!pip install tensorflow-io
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtype

In [3]:
!pip uninstall tensorflow
!pip install tensorflow==2.15.0

Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.16.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.16.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtype

In [4]:
import pandas as pd
import numpy as np

import os
import time
import torch
import tensorflow as tf
import keras
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve, roc_auc_score
import transformers
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, RobertaTokenizer, TFRobertaForSequenceClassification, AlbertTokenizer, TFAlbertForSequenceClassification

In [5]:
def load_data(filepath):
  data = pd.read_csv(filepath)
  print(data.head())

  return data

In [6]:
data = load_data("/content/drive/MyDrive/MDS_FYP/Data/suicidal_ideation_reddit_annotated.csv")

                                            usertext  label
0  I just want to end my life so badly. My life i...      1
1  My relationship is complicated and painful, bu...      1
2  I owe a lot of money , so I have to work.The c...      1
3  On the 2 of October I overdosed I just felt so...      1
4  Everyone tells me how wonderful I am, but not ...      1


In [7]:
data.shape

(12656, 2)

In [8]:
data["label"].value_counts()

label
1    6609
0    6047
Name: count, dtype: int64

In [9]:
data = data.dropna(subset=["usertext"])

In [10]:
def OOS_performance_measure(model, tokenizer, OOS_set):
    encodings = {
        "input_ids_OOS": [],
        "attention_masks_OOS": []
    }

    for text_row in OOS_set["usertext"]:
        # text_str = " ".join(text_row)
        tokens = tokenizer(text_row, truncation=True, padding="max_length", max_length=64, return_tensors="tf")
        encodings["input_ids_OOS"].append(tokens["input_ids"])
        encodings["attention_masks_OOS"].append(tokens["attention_mask"])

    input_ids_OOS = np.array(encodings["input_ids_OOS"])
    attention_masks_OOS = np.array(encodings["attention_masks_OOS"])
    input_ids_OOS = np.squeeze(input_ids_OOS, axis=1)
    attention_masks_OOS = np.squeeze(attention_masks_OOS, axis=1)

    OOS_labels = np.array(OOS_set["label"])

    OOS_predictions = model.predict([input_ids_OOS, attention_masks_OOS])
    OOS_proba = OOS_predictions.logits[:, 0]
    OOS_probabilities = tf.nn.sigmoid(OOS_proba).numpy()
    OOS_predictions_class = (OOS_proba > 0.5).astype(int)

    accuracy_OOS = accuracy_score(OOS_labels, OOS_predictions_class)
    precision_OOS = precision_score(OOS_labels, OOS_predictions_class)
    recall_OOS = recall_score(OOS_labels, OOS_predictions_class)
    f1_OOS = f1_score(OOS_labels, OOS_predictions_class)
    roc_auc_OOS = roc_auc_score(OOS_labels, OOS_probabilities)

    return accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS

In [11]:
MODEL_FILE_DistilBERT = "/content/drive/MyDrive/MDS_FYP/DistilBert_Intermediate/Model/distilbert"
loaded_model_DistilBert = TFDistilBertForSequenceClassification.from_pretrained(MODEL_FILE_DistilBERT)
# loaded_tokenizer_DistilBert = DistilBertTokenizer.from_pretrained(output_vocab_file)
MODEL_NAME = "distilbert-base-uncased"
loaded_tokenizer_DistilBert = DistilBertTokenizer.from_pretrained(MODEL_NAME)

accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS = OOS_performance_measure(loaded_model_DistilBert, loaded_tokenizer_DistilBert, data)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/MDS_FYP/DistilBert_Intermediate/Model/distilbert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

395/395 [==============================] - 40s 65ms/step


In [12]:
def save_result_to_csv(accuracy, precision, recall, f1_score, roc_auc, csv_filepath):
    if os.path.exists(csv_filepath):
        existing_df = pd.read_csv(csv_filepath)
    else:
        existing_df = pd.DataFrame()

    new_row = pd.DataFrame({
        "Accuracy": [accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F1_Score": [f1_score],
        "ROC AUC": [roc_auc]
    })

    existing_df = pd.concat([existing_df, new_row], ignore_index=True)
    existing_df.to_csv(csv_filepath, index=False)

    return existing_df

In [13]:
CSV_FILE_PATH = "/content/drive/MyDrive/MDS_FYP/OOS/OOS_metric_results_distilbert.csv"
OOS_eval = save_result_to_csv(accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS, CSV_FILE_PATH)

print("Holdout Accuracy:", accuracy_OOS)
print("Holdout Precision:", precision_OOS)
print("Holdout Recall:", recall_OOS)
print("Holdout F1-score:", f1_OOS)
print("Holdout ROC-AUC:", roc_auc_OOS)

Holdout Accuracy: 0.8161712247324614
Holdout Precision: 0.7770966479718273
Holdout Recall: 0.9071254567600487
Holdout F1-score: 0.8370916754478398
Holdout ROC-AUC: 0.883043808578639


In [14]:
MODEL_FILE_RoBERTa = "/content/drive/MyDrive/MDS_FYP/RoBERTa_Intermediate/Model/roberta"
loaded_model_RoBERTa = TFRobertaForSequenceClassification.from_pretrained(MODEL_FILE_RoBERTa)
# loaded_tokenizer_DistilBert = DistilBertTokenizer.from_pretrained(output_vocab_file)
MODEL_NAME = "roberta-base"
loaded_tokenizer_RoBERTa = RobertaTokenizer.from_pretrained(MODEL_NAME)

accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS = OOS_performance_measure(loaded_model_RoBERTa, loaded_tokenizer_RoBERTa, data)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/MDS_FYP/RoBERTa_Intermediate/Model/roberta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

395/395 [==============================] - 66s 128ms/step


In [15]:
CSV_FILE_PATH = "/content/drive/MyDrive/MDS_FYP/OOS/OOS_metric_results_roberta.csv"
OOS_eval = save_result_to_csv(accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS, CSV_FILE_PATH)

print("Holdout Accuracy:", accuracy_OOS)
print("Holdout Precision:", precision_OOS)
print("Holdout Recall:", recall_OOS)
print("Holdout F1-score:", f1_OOS)
print("Holdout ROC-AUC:", roc_auc_OOS)

Holdout Accuracy: 0.755132778438367
Holdout Precision: 0.8025743607583927
Holdout Recall: 0.7024969549330086
Holdout F1-score: 0.7492084111390761
Holdout ROC-AUC: 0.8451694345370521


In [16]:
MODEL_FILE_ALBERT = "/content/drive/MyDrive/MDS_FYP/Albert_Intermediate/Model/albert"
loaded_model_ALBERT = TFAlbertForSequenceClassification.from_pretrained(MODEL_FILE_ALBERT)
# loaded_tokenizer_DistilBert = DistilBertTokenizer.from_pretrained(output_vocab_file)
MODEL_NAME = "albert-base-v2"
loaded_tokenizer_ALBERT = AlbertTokenizer.from_pretrained(MODEL_NAME)

accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS = OOS_performance_measure(loaded_model_ALBERT, loaded_tokenizer_ALBERT, data)

Some layers from the model checkpoint at /content/drive/MyDrive/MDS_FYP/Albert_Intermediate/Model/albert were not used when initializing TFAlbertForSequenceClassification: ['dropout_4']
- This IS expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/MDS_FYP/Albert_Intermediate/Model/albert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForSequenceClassification for predictions 

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

395/395 [==============================] - 65s 137ms/step


In [17]:
CSV_FILE_PATH = "/content/drive/MyDrive/MDS_FYP/OOS/OOS_metric_results_albert.csv"
OOS_eval = save_result_to_csv(accuracy_OOS, precision_OOS, recall_OOS, f1_OOS, roc_auc_OOS, CSV_FILE_PATH)

print("Holdout Accuracy:", accuracy_OOS)
print("Holdout Precision:", precision_OOS)
print("Holdout Recall:", recall_OOS)
print("Holdout F1-score:", f1_OOS)
print("Holdout ROC-AUC:", roc_auc_OOS)

Holdout Accuracy: 0.6409829567974633
Holdout Precision: 0.6481616044179626
Holdout Recall: 0.6790499390986602
Holdout F1-score: 0.6632463380176965
Holdout ROC-AUC: 0.6872840454805202
